In [ ]:
# basic package
import csv
import glob
import pandas as pd
import matplotlib as plt

import numpy as np
import random
from operator import itemgetter
import matplotlib.pyplot as plt




# ml related 
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

Functionality.ipynb                 political_selected.csv
Troll full data.pdf                 sued_full_dataset.pdf
drive-download-20220521T173459Z-001 undernourish_selected.csv


In [ ]:
political = pd.read_csv("political_selected.csv")
undernourish = pd.read_csv("undernourish_selected.csv")


# Function Definitions for Random forest


In [ ]:
# functions you will need 

def map_code_to_meaning(mapping, code_no):
    return (mapping.loc[mapping['code']==code_no]).iloc[0].var_name

# print the total percetnage of missing in each dataset
def total_percentage_missing(df):
    return(np.count_nonzero(df.isna()) / df.size)

# drop the top N rows with most NAs
def drop_top_N_rows_with_most_NAs(df, N=300):
    if N/len(df)> 0.2:
      warnings.warn("Based on your speficied N, you are dropping more then 20% of the data")

    print("shape before drop", df.shape)
    dict_nas = {}
    for i in range(len(df)):
        percentage = total_percentage_missing(df.iloc[i])
        dict_nas[i] = percentage
    res = dict(sorted(dict_nas.items(), key = itemgetter(1), reverse = True)[:N])
    # print("here")
    top_NAs_rows = list(res.keys())
    # print(top_NAs_rows)
    df.drop(top_NAs_rows, axis=0, inplace=True)
    print("shape after drop ", df.shape)
    print("Missing data percentage ", total_percentage_missing(df) )
    return df

# split into two dataset by year (default=2017)
# fist one include that year, second one is year after that
def split_by_year(df, split_at = 2017):
    res1 = df.loc[df['Year']<= split_at]
    res2 = df.loc[df['Year']>= split_at]
    return res1, res2

def print_all_coeff(list_coef, feature_name):
    sort_index = reversed(np.argsort(list_coef))
    list_of_lists = []
    for i in sort_index:
#         print(feature_name[i])
        temp = int(feature_name[i])
        if list_coef[i] !=0.0:
            list_of_lists.append([round(list_coef[i],10), feature_name[i], map_code_to_meaning(meaning_map, temp)])
    return pd.DataFrame(list_of_lists, columns =['non_zero_coefficient', 'code', 'variable_name'])

  # pipeline on returning the coefficient of lasso regression
# also returns the score of the regressions
def lasso_pipeline_for_cross_validation(df, target_name = 'political', split_year = 2017, lasso_alpha = 0.12):

    if target_name not in df.columns:
        raise ValueError("The input dataframe doesn't have the column: political")
    
    if 'Continent' in df.columns:
      df = df.drop(columns =['Continent'])

    # default split at 2017
    political_pre_2017, political_post_2017 = split_by_year(df, split_at = split_year)
    
    # Note, the variable names here is only names, y_politcal can be any dataframe
    # doesn't have to be political 
    y_political = political_pre_2017.pop(target_name)
    X_political = political_pre_2017.drop(columns = ['Year', 'Area Code'])

    y_political_test = political_post_2017.pop(target_name)
    X_political_test = political_post_2017.drop(columns = ['Year', 'Area Code'])

    
    feature_names = X_political_test.columns

    # scale the X
    scaler = StandardScaler()
    political_scaler_X = scaler.fit(X_political)
    X_political_scaled = political_scaler_X.transform(X_political)
    X_political_test_scaled = political_scaler_X.transform(X_political_test)

    # scale the y
    y_political = y_political.values.reshape(-1,1)
    y_political_test = y_political_test.values.reshape(-1,1)
    political_scaler_y = scaler.fit(y_political)
    y_political_scaled = political_scaler_y.transform(y_political)
    y_political_test_scaled = political_scaler_y.transform(y_political_test)
    
    # Run LASSO
    reg = linear_model.Lasso(alpha=lasso_alpha).fit(X_political_scaled, y_political_scaled)
    
    
    y_train_pred = reg.predict(X_political_scaled) # predicting for training
    y_pred = reg.predict(X_political_test_scaled)  # predicting for testing
    
    
    r2_training = r2_score(y_political_scaled,y_train_pred)
    r2_resting = r2_score(y_political_test_scaled, y_pred)
    
   
    MAE_train = mean_absolute_error(y_political_scaled,y_train_pred)
    MAE_test = mean_absolute_error(y_political_test_scaled, y_pred)
    
    return MAE_train, r2_training, MAE_test, r2_resting

In [ ]:
political = drop_top_N_rows_with_most_NAs(df= political, N= 300)
undernourish = drop_top_N_rows_with_most_NAs(df= undernourish, N= 300)




# fill NAs
political = political.fillna(0)
undernourish = undernourish.fillna(0)


shape before drop (3705, 1002)
shape after drop  (3405, 1002)
Missing data percentage  0.003907896395168547
shape before drop (3933, 1002)
shape after drop  (3633, 1002)
Missing data percentage  0.02336230374373741


# Random Forest

In [ ]:
def findElements(lst1, lst2):
    return [lst1[i] for i in lst2]

def reandom_forest(df, target_name = 'political', split_year = 2017):

    if target_name not in df.columns:
        raise ValueError("The input dataframe doesn't have the column: political")
    
    if 'Continent' in df.columns:
        df = df.drop(columns =['Continent'])

    # default split at 2017
    political_pre_2017, political_post_2017 = split_by_year(df, split_at = split_year)
    
    # Note, the variable names here is only names, y_politcal can be any dataframe
    # doesn't have to be political 
    y_political = political_pre_2017.pop(target_name)
    X_political = political_pre_2017.drop(columns = ['Year', 'Area Code'])

    y_political_test = political_post_2017.pop(target_name)
    X_political_test = political_post_2017.drop(columns = ['Year', 'Area Code'])

    
    feature_names = X_political_test.columns

    # scale the X
    scaler = StandardScaler()
    political_scaler_X = scaler.fit(X_political)
    X_political_scaled = political_scaler_X.transform(X_political)
    X_political_test_scaled = political_scaler_X.transform(X_political_test)

    # scale the y
    y_political = y_political.values.reshape(-1,1)
    y_political_test = y_political_test.values.reshape(-1,1)
    political_scaler_y = scaler.fit(y_political)
    y_political_scaled = political_scaler_y.transform(y_political)
    y_political_test_scaled = political_scaler_y.transform(y_political_test)
    
    # print shapes
    print("Training Shape:", X_political_scaled.shape)
    print("Testing Shape", X_political_test_scaled.shape)
    
    # Run LASSO
    random.seed(1234)
    rf = RandomForestRegressor(n_estimators=80)
    print(y_political_scaled.shape)
    rf.fit(X_political_scaled, y_political_scaled.ravel())
    
    std = np.std([rf.feature_importances_ for tree in rf.estimators_],
             axis=0)
    
    y_train_pred = rf.predict(X_political_scaled)
    y_test_pred = rf.predict(X_political_test_scaled)
    print("Mean Absolute Error on training", mean_absolute_error(y_political_scaled, y_train_pred))
    print("Mean Absolute Error on testing",  mean_absolute_error(y_political_test_scaled, y_test_pred))
    return rf.feature_importances_, feature_names, std
    

#     # evaluation
#     print("score on training dataset", reg.score(X_political_scaled, y_political_scaled) )
#     print("score on testing dataset", reg.score(X_political_test_scaled, y_political_test_scaled))
#     y_train_pred = reg.predict(X_political_scaled)
#     y_pred = reg.predict(X_political_test_scaled)
#     print("R squared score on training", r2_score(y_political_scaled, y_train_pred))
#     print("R squared score on testing", r2_score(y_political_test, y_pred))
   

#     res_df = print_all_coeff(reg.coef_, feature_names)
#     return res_df

In [ ]:
print("hi")
res, names , stds = reandom_forest(political, target_name = 'political')

hi
Training Shape: (3041, 999)
Testing Shape (546, 999)
(3041, 1)
Mean Absolute Error on training 0.06150192643695166
Mean Absolute Error on testing 0.13487458937446906


In [ ]:
N = 50
import_f = sorted(range(len(res)), key = lambda sub: res[sub])[-N:]

important_code = findElements(names,import_f )
important_stds = findElements(list(stds), import_f)

meanings_import = []

for each in important_code: 
    
    meanings_import.append(each)
 

In [ ]:
political_filterd = political.filter(items = meanings_import)

In [ ]:
meanings_import

['69897230',
 '6990723112',
 '17385312',
 '68177230',
 '680072184',
 '69917273',
 '69907273',
 '69745008',
 '1664',
 '69785008',
 '6990723114',
 '6990723113',
 '69917225',
 '68217266',
 '68187225',
 '69725008',
 '65157225',
 '68217225',
 '65157273',
 '19545622',
 '17355312',
 '68187266',
 '6996726313',
 '66465110',
 '69815008',
 '6818723113',
 '6821726313',
 '69835007',
 '68007273',
 '66507208',
 '188294',
 '230826110',
 '17527213',
 '2041152',
 '17525118',
 '66505110',
 '3010512',
 '66217208',
 '68177266',
 '68187265',
 '65177225',
 '65077265',
 '69705008',
 '3010511',
 '2264',
 '3010551',
 '66705110',
 '202972380',
 '4635510',
 '220086185']

# Undernourish

In [ ]:
res, names , stds = reandom_forest(undernourish, target_name = 'undernourish')

Training Shape: (3055, 999)
Testing Shape (771, 999)
(3055, 1)
Mean Absolute Error on training 0.04151812080248604
Mean Absolute Error on testing 0.1726539659023384


In [ ]:
N = 50
import_f = sorted(range(len(res)), key = lambda sub: res[sub])[-N:]

important_code = findElements(names,import_f )
important_stds = findElements(list(stds), import_f)

meanings_import = []

for each in important_code: 
    
    meanings_import.append(each)
 

In [ ]:
meanings_import

['17385419',
 '622465',
 '67515110',
 '17735922',
 '67607246',
 '65057273',
 '188295',
 '1802465',
 '185610',
 '180294',
 '17735910',
 '17805420',
 '1890462',
 '173464',
 '5155610',
 '25764',
 '1664',
 '17205419',
 '65664',
 '184795',
 '622462',
 '67917245',
 '220306161',
 '4635312',
 '65157230',
 '184864',
 '67917225',
 '220306192',
 '68247273',
 '4065622',
 '67937245',
 '67917230',
 '104164',
 '16305510',
 '2203061930',
 '17805318',
 '67917246',
 '65177230',
 '6575622',
 '17175419',
 '18955910',
 '12325910',
 '6824723113',
 '65057225',
 '10165111',
 '65965',
 '184794',
 '8825420',
 '4635510',
 '17355419']